In [1]:
# Libraries, Variables and Constants

import autogen, os
import json
from dotenv import load_dotenv

load_dotenv("./../credentials_my.env")
cache_seed = 41 # default seed is 41

In [2]:
my_models = [
    {
        'model': os.environ['GPT4-0125PREVIEW-128k'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY_SCUS'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT_SCUS'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    },
    {
        'model': os.environ['GPT4-1106-128k'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    },
    {
        'model': os.environ['GPT432-0613-32k'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    }
]

# need for converting to string
os.environ['models_var'] = json.dumps(my_models)

# Setting configurations for autogen
config_list = autogen.config_list_from_json(
    env_or_file='models_var',
    filter_dict={
        "model": {"gpt4-0125preview-128k", "gpt-4"}
    }
)

# config_list

In [3]:
# Setting configurations for autogen
config_list = autogen.config_list_from_json(
    env_or_file='models_list.json',
    filter_dict={ "model": {"gpt4-0125preview-128k", "gpt-4"} }
)

# config_list

In [4]:
autogen.Completion.clear_cache()

llm_config={
        # "use_cache": False,
        "cache_seed": cache_seed,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling
    }

# llm_config

In [5]:
# create a UserProxyAgent instance named "user_proxy"

user_proxy = autogen.UserProxyAgent(
    name                       = "user_proxy",
    human_input_mode           = "NEVER", # NEVER / ALWAYS / TERMINATE
    max_consecutive_auto_reply = 10,

    # if the x["content"] ends by "TERMINATE", is_termination_msg-->True; otherwise, is_termination_msg--> False
    is_termination_msg         = lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    
    code_execution_config = {
        "work_dir": "coding",
        
        # Using docker is safer than running the generated code directly.
        # set use_docker=True if docker is available to run the generated code. 
        "use_docker": False
    },
    
    llm_config=llm_config
)

user_proxy

In [6]:
# create an AssistantAgent named "assistant"

assistant = autogen.AssistantAgent(
    name       = "assistant",
    llm_config = llm_config
)

assistant

In [ ]:
%%time
# the assistant receives a message from the user_proxy, which contains the task description in the "message" field

chat_res = user_proxy.initiate_chat(
    assistant,
    message = """What date is today? Compare the year-to-date gain for META and TESLA.""",
    summary_method = "reflection_with_llm",
)

user_proxy (to assistant):

What date is today? Compare the year-to-date gain for META and TESLA.

--------------------------------------------------------------------------------


## Only if needed:

In [ ]:
%%time

# followup of the previous question, if needed
user_proxy.send(
    recipient=assistant,
    message="""Please try again and use another way to gather the stock price. For example, use yfinance.""",
)

## Ask an additional sub-question

In [ ]:
%%time

user_proxy.send(
    recipient=assistant,
    message="""
    Plot a chart of their stock prince change YTD. Save teh data to stock_price_ytd.csv, and 
    save the plot to stock_price_ytd.png.
    """,
)

## Let's check the History

In [ ]:
i = 1
for cr in chat_res.chat_history:
    print(f'Step {i} --> {cr["content"].rstrip()[-9:]}')
    i=i+1

In [ ]:
# read the content of the cache

import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect(f'./.cache/{cache_seed}/cache.db')

# Create a cursor object
cur = conn.cursor()

# List all tables in the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(f"Tables in this DB: {cur.fetchall()}\n")

# Execute a query
cur.execute("SELECT * FROM Cache")

# Fetch and print the results
for row in cur.fetchall():
    print(row)

# Close the connection
conn.close()